# init

In [36]:
import turicreate


products = turicreate.SFrame('amazon_baby.sframe')
int_columns = ['rating']

for col in int_columns:
    products[col] = products[col].apply(int)
products = products[products['rating']!= 3]    
products['sentiment'] = products['rating'] >= 4
products['word_count'] = turicreate.text_analytics.count_words(products['review'])
train_data,test_data = products.random_split(.8,seed=0)
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)


selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

def select_words(words_count,selected_words):
    d = {}
    for word in selected_words:
        if word not in words_count: d[word] = 0
        else: d[word] = words_count[word]
    return d

products['selected_word_count'] = products['word_count'].apply(lambda wc: select_words(wc, selected_words))

for word in selected_words:
    products[word] = products['word_count'].apply(lambda word_count: word_count[word] if word in word_count else 0)

train_data_words,test_data_words = products.random_split(.8,seed=0)
selected_words_model = turicreate.logistic_classifier.create(train_data_words,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data_words)


Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 4         | 5        | 0.471960     | 0.847708          | 0.846385            |

| 5         | 6        | 0.559347     | 0.847708          | 0.846385            |

| 6         | 7        | 0.643465     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

### 1. Das 11 palavras em selected_words, qual é a mais usada nas avaliações do conjunto de dados?

### 2. Das 11 palavras em selected_words, qual é a menos usada nas resenhas do conjunto de dados?  

In [38]:
# Calcular a soma de ocorrências para cada palavra em selected_words
word_counts = {word: products[word].sum() for word in selected_words}

# Encontrar a palavra mais usada
max_word = max(word_counts, key=word_counts.get)
# Encontrar a palavra menos usada
min_word = min(word_counts, key=word_counts.get)

# Imprimir os resultados
print('Most used word:', max_word, 'with count:', word_counts[max_word])
print('Least used word:', min_word, 'with count:', word_counts[min_word])

Most used word: great with count: 55791.0
Least used word: wow with count: 425


### 3. Das 11 palavras em selected_words, qual delas obteve o peso mais positivo no selected_words_model? 

### 4. Das 11 palavras em selected_words, qual delas obteve o peso mais negativo no selected_words_model?

In [39]:
words_coefficients = selected_words_model.coefficients.sort(key_column_names='value')
# Encontrar palavras com maior e menor valor usando lambda
most_positive = max(words_coefficients, key=lambda x: x['value'])
most_negative = min(words_coefficients, key=lambda x: x['value'])

print('most positive: ', most_positive['name'], most_positive['value'])
print('most negative: ', most_negative['name'], most_negative['value'])

most positive:  love 1.359268866922504
most negative:  horrible -2.251335236759102


### 5. Qual dos seguintes intervalos contém a precisão do modelo selected_words_model nos dados de teste?

In [52]:
accuracy_selected_word = selected_words_model.evaluate(test_data_words)
print(f"+Accuracy: {accuracy_selected_word.get('accuracy')}")


+Accuracy: 0.8463848186404036


### 6. Qual dos intervalos a seguir contém a precisão do sentiment_model no IPython Notebook a partir de uma aula sobre test_data?

In [53]:
accuracy_sentiment_model = sentiment_model.evaluate(test_data)
print(f"+Accuracy: {accuracy_sentiment_model.get('accuracy')}")


+Accuracy: 0.9176975738650012


### 7 Qual dos intervalos a seguir contém a precisão do classificador de classe majoritária, que simplesmente prevê a classe majoritária nos dados de teste
.835


### 8. Como o senhor compara os diferentes modelos aprendidos com a abordagem de linha de base em que estamos apenas prevendo a classe majoritária? 
O modelo aprendido usando todas as palavras teve um desempenho muito melhor do que os outros dois.  As outras duas abordagens tiveram o mesmo desempenho.

### 9. Qual dos intervalos a seguir contém o "predicted_sentiment" para a avaliação mais positiva de "Baby Trend Diaper Champ", de acordo com o sentiment_model do IPython Notebook da palestra?

In [57]:
diaper_champ_reviews = products[products['name']=='Baby Trend Diaper Champ']
sentiment_model.predict(diaper_champ_reviews[0], output_type='probability')

dtype: float
Rows: 1
[0.9950122935569565]

### 10. Considere a avaliação mais positiva de "Baby Trend Diaper Champ" de acordo com o sentiment_model do IPython Notebook da palestra. Qual dos seguintes intervalos contém o predicted_sentiment para essa avaliação, se usarmos o selected_words_model para analisá-la?

In [58]:
selected_words_model.predict(diaper_champ_reviews[0], output_type='probability')

dtype: float
Rows: 1
[0.7919288370624482]

### 11. Por que o valor do predicted_sentiment para a avaliação mais positiva encontrada usando o sentiment_model é muito mais positivo do que o valor previsto usando o selected_words_model?
Nenhuma das selected_words apareceu no texto desta revisão. 